# The problem

http://dasl.datadesk.com/data/view/15

In May, 1978, Brink's Inc. was awarded a contract to collect coins from some 70,000 parking meters in New York City for delivery to the City Department of Finance. Sometime later the City became suspicious that not all of the money collected was being returned to the city. In April of 1978 five Brink's collectors were arrested and charged with grand larceny. They were subsequently convicted. The city sued Brink's for negligent supervision of its employees, seeking to recover the amount stolen. As the fact of theft had been established, a reasonable estimate of the amount stolen was acceptable to the judge.

## Let's take a look at the data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

In [ ]:
%matplotlib inline

In [ ]:
meter_records = pd.read_csv("data/parking-meters.tsv", sep="\t")
meter_records.head()

This didn't work well at all. This file must be poorly formatted.

In [ ]:
meter_records = pd.read_csv("data/parking-meters.tsv", sep="\t", skiprows=1, header=None, 
                            names=["month", "total", "city", "brinks"])
meter_records.head(10)

In [ ]:
meter_records.dtypes

Look at line 9. For some reason, we have a space in the number. I could remove that space, but judging from the other numbers, I'm not sure that's good data. I'm going to throw it out.

In [ ]:
meter_records = pd.read_csv("data/parking-meters.tsv", sep="\t", skiprows=[0,11], header=None, 
                            names=["month", "total", "city", "brinks"])
meter_records.head(10)

In [ ]:
meter_records.dtypes

I want a point of comparison each month to see the amount of money taken in, but each month has differing amounts of parking, so a simple comparison of the total doesn't make sense. If we compare the amount taken in to the amount taken in by city workers, that could work, as both should track. Let's verify that.

In [ ]:
meter_records.city.corr(meter_records.total)

In [ ]:
meter_records.plot(kind="scatter", x="city", y="total", figsize=(10, 6))
m, b = np.polyfit(meter_records.city, meter_records.total, 1)
plt.plot(meter_records.city, m*meter_records.city + b)

This is a medium correlation -- not great, but not bad. I don't have anything else to use, so let's go with it.

In [ ]:
meter_records['adj_revenue'] = meter_records['total'] / meter_records['city']

In [ ]:
meter_records.head()

Now let's see the mean adjusted revenue for months when Brinks was active (1) and not active (0).

In [ ]:
meter_records.pivot_table(columns=['brinks'], values=['adj_revenue'])

## The p-value

There's definitely a difference, but is it random chance or is this actually significant? In order to find out, we want the _p-value_.

What is a _p-value_?

For a test of statistical significance, we start with the _null hypothesis_. This is the hypothesis that there's no relation between groups. It's generally the opposite of what we're testing for. In this case, the null hypothesis is that there's no relation between whether Brinks was operating the meters and the amount of revenue brought in.

Once you have that, you compare your observed result -- in this case the difference in mean adjusted revenue for months when Brinks was active and when it was inactive -- to some statistical model to see how extreme your result is. Another way of looking at it is, given your observed result, what's the likelihood the null hypothesis is true?

People argue about the next step, but it's an OK rule of thumb: p-values of < 0.05 mean the null hypothesis is most likely not true.

Note that we never see the chance that our alternative hypothesis -- the thing we're testing for -- is true. We are accepting or rejecting the null hypothesis.

### Shuffling labels

There are complex formula-based ways to calculate the p-value. I don't know them. I have a computer, though, so I can use another way. We can shuffle the labels -- in this case, shuffle the "brinks" value to different months -- many times and record our results. Shuffling them assume they don't matter -- our null hypothesis. If we do this many times, we can get a distribution of results and then see where our observed result falls on that. I'm going to do this with Pandas, but there's other ways.

In [ ]:
# Get our observed value.
def mean_revenue_diff(df):
    revenues = df.pivot_table(columns=['brinks'], values=['adj_revenue'])
    return (revenues[0] - revenues[1])['adj_revenue']

observed_mean_diff = mean_revenue_diff(meter_records)
observed_mean_diff

In [ ]:
# Copy the table so we can mess with it.
mr2 = meter_records.copy()
mr2.head(10)

In [ ]:
# Shuffle the Brinks column.
mr2.brinks = np.random.permutation(mr2.brinks)
mr2.head(10)

In [ ]:
mr2.pivot_table(columns=['brinks'], values=['adj_revenue'])

In [ ]:
mean_revenue_diff(mr2)

In [ ]:
num_experiments = 10000
results = []
count = 0
for _ in range(num_experiments):
    mr2.brinks = np.random.permutation(mr2.brinks)
    mean_diff = mean_revenue_diff(mr2)
    results.append(mean_diff)
    if observed_mean_diff >= 0 and mean_diff >= observed_mean_diff:
        count += 1
    elif observed_mean_diff < 0 and mean_diff <= observed_mean_diff:
        count += 1

In [ ]:
print("Observed difference of two means: %.2f" % observed_mean_diff)
print(count, "out of", num_experiments, "experiments had a difference of two means ", end="")
if observed_mean_diff < 0:
    print("less than or equal to ", end="")
else:
    print("greater than or equal to ", end="")
print("%.2f" % observed_mean_diff, ".")
print("The chance of getting a difference of two means ", end="")
if observed_mean_diff < 0:
    print("less than or equal to ", end="")
else:
    print("greater than or equal to ", end="")
print("%.2f" % observed_mean_diff, "is", (count / float(num_experiments)), ".")

That's our p-value! Let's see it on a graph.

In [ ]:
plt.figure(figsize=(10, 6))
sns.distplot(results, kde=False)
plt.vlines(observed_mean_diff, 0, 120, colors="g", linestyle="dashed")

## Confidence intervals

We'd like to know the amount of money Brinks owes the city, but there's not a good way to say exactly what that is. (Can you think of a way?)

Even though our results were statistically significant, they might not even be important. What if Brinks employees stole $100/month? To NYC, the costs of taking the case to trial would dwarf that. _Confidence intervals_ show us importance. A confidence interval is simply the range of likely results. In general, this is the middle 90% of possible values.

How do we get "possible values?" We can use a technique called "bootstrapping". We create samples of the data the same size as the original, taking observations randomly and _with replacement_. This means that we might pick the same observation more than once -- which is what we want. We do this 10,000 times, taking the difference of means each time.

In [ ]:
meter_records.sample(n=10, replace=True)

In [ ]:
conf_interval = 0.9
num_experiments = 1000
results = []
for _ in range(num_experiments):
    df = meter_records.sample(frac=1, replace=True)
    mean_diff = mean_revenue_diff(df)
    results.append(mean_diff)

In [ ]:
results.sort()
tails = (1 - conf_interval) / 2
lower_bound = int(math.ceil(num_experiments * tails))
upper_bound = int(math.floor(num_experiments * (1 - tails)))

In [ ]:
print("Observed difference between the means: %.2f" % observed_mean_diff)
print("We have %d%% confidence that the true difference between the means is between: %.2f and %.2f" % \
      (conf_interval * 100, results[lower_bound], results[upper_bound]))

This doesn't get us an amount of dollars, though. How could we do that?

In [ ]:
# Do amount of dollars calculations here.

## Resources

https://speakerdeck.com/jakevdp/statistics-for-hackers
http://www.amazon.com/Statistics-Edition-Synthesis-Lectures-Mathematics/dp/160845570X/ref=dp_ob_title_bk